# 空间关系及操作

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas

pd.options.display.max_rows = 10

In [ ]:
countries = geopandas.read_file("zip://./data/ne_110m_admin_0_countries.zip")
cities = geopandas.read_file("zip://./data/ne_110m_populated_places.zip")
rivers = geopandas.read_file("zip://./data/ne_50m_rivers_lake_centerlines.zip")

## 空间关系

地理信息数据一个重要的方面就是我们可以看到 *空间关系*: 两个空间对象如何相互联系 (相交, 相切, 包含, 等等).

拓扑学, GIS 中的集合理论关系都是基于 DE-9IM 模型. 看 https://en.wikipedia.org/wiki/Spatial_relation 了解更多.

![](img/TopologicSpatialRelations2.png)
(Image by [Krauss, CC BY-SA 3.0](https://en.wikipedia.org/wiki/Spatial_relation#/media/File:TopologicSpatialRelarions2.png))

### 个体联系

我们先来创造一些简单的空间对象:

一个多边形 <small>(提醒: 我们在这里使用 `.squeeze()` 来从 GeoSeries 里提取标量几何对象)</small>:

In [ ]:
belgium = countries.loc[countries['name'] == 'Belgium', 'geometry'].squeeze()

两个点:

In [ ]:
paris = cities.loc[cities['name'] == 'Paris', 'geometry'].squeeze()
brussels = cities.loc[cities['name'] == 'Brussels', 'geometry'].squeeze()

一条线段:

In [ ]:
from shapely.geometry import LineString
line = LineString([paris, brussels])

我们把这四个几何对象放在一起可视化 (只是简单的把他们放在一个 GeoSeries 里然后使用 geopandas `.plot()` 方法):

In [ ]:
geopandas.GeoSeries([belgium, paris, brussels, line]).plot(cmap='tab10')

你可以认出这是抽象的比利时的形状.

布鲁塞尔, 比利时首都, 所以在比利时境内. 这是一个空间联系, 我们可以使用这两个个体几何图形进行如下测试:

In [ ]:
brussels.within(belgium)

使用一个相反的操作, 比利时包含布鲁塞尔:

In [ ]:
belgium.contains(brussels)

另一方面, 巴黎不在比利时境内:

In [ ]:
belgium.contains(paris)

In [ ]:
paris.within(belgium)

我们画的从巴黎到布鲁塞尔的线并不完全在比利时境内, 但是它和比利时相交:

In [ ]:
belgium.contains(line)

In [ ]:
line.intersects(belgium)

### GeoDataFrames 的空间联系

相同的方法也可以使用在个别 `shapely` 几何图形上, 也可以作为 `GeoSeries` / `GeoDataFrame` 对象的方法使用.

例如, 如果我们对 `paris` 点在世界数据集上调用 `contains` 方法, 它会对 `world` dataframe 中的每个国家做空间检查:

In [ ]:
countries.contains(paris)

因为上面的语句会返回一个布尔值, 所以我们可以用来过滤数据:

In [ ]:
countries[countries.contains(paris)]

确实, 巴黎只坐落在法国.

另一个例子, 提取南美亚马逊河的线段, 我们可以查询河流流过哪些城市:

In [ ]:
amazon = rivers[rivers['name'] == 'Amazonas'].geometry.squeeze()

In [ ]:
countries[countries.crosses(amazon)]  # 或者 .intersects

<div class="alert alert-info" style="font-size:120%">
<b>参考</b>: <br><br>

空间联系检查函数一览 (*空间断言函数*):

<ul>
  <li>`equals`</li>
  <li>`contains`</li>
  <li>`crosses`</li>
  <li>`disjoint`</li>
  <li>`intersects`</li>
  <li>`overlaps`</li>
  <li>`touches`</li>
  <li>`within`</li>
  <li>`covers`</li>
</ul>

<p>
函数简介： https://shapely.readthedocs.io/en/stable/manual.html#predicates-and-relationships.
<p></p>
更多详情见：https://en.wikipedia.org/wiki/DE-9IM.
</p>
</div>

## 空间操作

除了对空间进行断言返回布尔值外, Shapely 和 GeoPandas 也提供分析方法返回一个新的几何对象.

详情请见： https://shapely.readthedocs.io/en/stable/manual.html#spatial-analysis-methods.

利用使用上面的简单数据, 我们建立一个环绕布鲁塞尔的缓冲区 (返回多边形):

In [ ]:
geopandas.GeoSeries([belgium, brussels.buffer(1)]).plot(alpha=0.5, cmap='tab10')

现在对这两个多边形取交叉, 联合或者相异。

In [ ]:
brussels.buffer(1).intersection(belgium)

In [ ]:
brussels.buffer(1).union(belgium)

In [ ]:
brussels.buffer(1).difference(belgium)

还有一个有用的方法 `unary_union` 属性, 可以通过联合操作把一个几何对象几何合并成单个的几何对象.

例如, 我们可以建立一个非洲大陆的几何对象:

In [ ]:
africa_countries = countries[countries['continent'] == 'Africa']

In [ ]:
africa = africa_countries.unary_union

In [ ]:
africa

In [ ]:
print(str(africa)[:1000])

<div class="alert alert-info" style="font-size:120%">
<b>记住</b>: <br><br>

GeoPandas (个别对象用 Shapely) 提供许多基础方法分析地理信息数据(距离, 长度, 质心, 边界, 凸包, 简化, 变形, ....), 比我们在教程里接触的要多的多.


<ul>
  <li>GeoPandas 提供了所有方法的介绍: http://geopandas.readthedocs.io/en/latest/reference.html</li>
</ul>

</div>

